In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
import json
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
import matplotlib.pyplot as plt

# Convert the json data in string (transformed by the pd.read_csv) to string 

In [ ]:
data = pd.read_csv("../bpideep/rawdata/data2020-12-03.csv")

In [ ]:
def load_json_field(bad_json):
    regex = r"\w\'\w"
    subst = ""
    bad_json = re.sub(regex, subst, bad_json)
    bad_json = bad_json.replace("d' Arrouzat", "darrouzat")
    good_json = bad_json.replace("\'", "\"").replace("None", "null").replace("True", "true").replace("False", "false")
    
    return json.loads(good_json)

In [ ]:
data["team"] = data["team"].apply(load_json_field)
data["industries"] = data["industries"].apply(load_json_field)
data["investors"] = data["investors"].apply(load_json_field)
data["fundings"] = data["fundings"].apply(load_json_field)

# TEST 

In [ ]:
import re
import math

def average_list(range_list):
    return sum(range_list)/len(range_list)

def compute_employees_mean(data):
    range_list = list(data.employees.unique())
    range_list.remove("n.a.")
    keys_list = []
    means_list = []

    for i in range(len(range_list)) :
        if type(range_list[i]) == str :
            temp = re.findall(r'\d+', range_list[i]) 
            res = list(map(int, temp)) 
            mean = average_list(res)
            means_list.append(mean)
            keys_list.append(range_list[i])
        else:
            pass

    zip_iterator = zip(keys_list, means_list)
    range_dict = dict(zip_iterator)
    
    return range_dict

def impute_missing_employees(data):
    
    data = data.copy()
    data["employees_imputed"] = data.employees_clean
    

    #imputing according to the range given by Dealroom in the column "employees" 
    #thanks to the function "compute_employees_mean" 
    range_dict = compute_employees_mean(data)
    data.loc[data.employees_clean.isna(), "employees_imputed"] = range_dict[data.employees]
    
    return data
    


In [ ]:
#hf['w'] = hf.apply(lambda x: gf.loc[(x.a, x.b)], axis=1)


def replace_employees(df):
    df["employees_imputed"] = df.employees_latest
    dictionary = compute_employees_mean(df)
    for key, value in dictionary.items():
        df.loc[(df.employees == key) & (df.employees_latest.isna()), "employees_imputed"] = value
    return df

In [ ]:
tmp = data

In [ ]:
tmp[["employees_latest","employees_imputed"]].isnull().sum()

In [ ]:
tmp[tmp.employees.isnull()][["employees"]].isna().sum()

In [ ]:
data.employees.unique()

In [ ]:
tmp.employees_imputed.isna().sum()

In [ ]:
tmp.employees_latest.isnull().sum()

In [ ]:
tmp.loc[tmp.employees_latest.isna(), "employees"]

In [ ]:
tmp["employees_imputed"] = tmp.employees_latest

In [ ]:
dictionary

In [ ]:
tmp[tmp.employees_latest.isna()]["employees_imputed"] = tmp["employees"].apply(lambda x: dictionary.get(x))


In [ ]:
tmp[tmp.name == "Biin"][["name", "employees", "employees_latest", "employees_imputed"]]

# Import data + data cleaning thanks to info retrieved from Linkedin

## DATA CLEANING

In [2]:
def impute_missing_launch_year(data):

    names = ['Amypore',
             'Kinnov Therapeutics',
             'Lipofabrik',
             'Step pharma',
             'LiMM Therapeutics',
             'Ilek',
             'LysPackaging',
             'TexiSense',
             "Institut de Prise en Charge de l'Obésité",
             'Izi Family',
             'Arthur Dupuy',
             'Gen.Orph',
             'Uniris',
             'NANOZ',
             'Akwatyx',
             'Black-line',
             'Eyye',
             "O'Sol",
             'Treenox',
             'Aqualeg',
             'Co-assit',
             'Wind my roof',
             'APPARTOO',
             'BimBamJob',
             'Buddytherobot.com',
             'Bcm',
             'CCI Paris Ile de France',
             'Datarocks',
             'EFFICIENCIA',
             'EONEF',
             'FEALINX',
             'INERIS',
             'Marguerite',
             'TokTokDoc',
             'Novaquark',
             'Peopeo',
             'Sloclap',
             'Swift',
             'Sword',
             'XT-VISION',
             'Ryax',
             'Sylha',
             'Opta LP']

    # inserting missing launch year and month for the 11 companies with fundraising
    launch_years = [2018, 2015, 2012, 2014, 2018, 2016, 2015, 2010, 2011, 2016, 2015, 
                   2012, 2017, 2012, 2015, 2017, 2016, 2016, 2018, 2011, 2016, 2018, 2015, 2015, 2014, 2014, 2013,
                   2014, 2012, 2016, 1997, 1990, 2012, 2016, 2014, 2017, 2015, 1973, 2000, 2011, 2017, 2019, 2013]

    zipbObj = zip(names, launch_years)
    launch_year_dict = dict(zipbObj)

    data["launch_year_clean"] = data['launch_year']
    
    for name, year in launch_year_dict.items() : 
        data.loc[data.name == name, "launch_year_clean"] = year
    return data

def get_growth_dict(df):
    table = df[["growth_stage", "launch_year_clean", "id"]].groupby(by=["launch_year_clean", "growth_stage"]).count()
    column = list(table.unstack(level = -1).columns)
    growth_dict = table.unstack(level = -1).fillna(0).apply(lambda x: column[x.argmax()][1], axis = 1).to_dict()
    
    return growth_dict
    

def fill_missing_growth(growth_dict, growth_stage, launch_year_clean):
   
    if type(growth_stage) == str:
        return growth_stage    
    elif launch_year_clean in growth_dict:
        return growth_dict[launch_year_clean]
    else:
        return growth_stage

## TEST cleaning

In [ ]:

tmp = pd.read_csv("../bpideep/rawdata/data2020-12-03.csv")
tmp.drop(data[(data.id == 1787891) | (data.id == 1893232)].index, inplace = True)

dico = get_growth_dict(tmp)


In [ ]:
impute_missing_launch_year(tmp)

In [ ]:
tmp['growth_stage_imputed'] = tmp.apply(lambda row: fill_missing_growth(dico,
                row['growth_stage'], 
                row['launch_year_clean']), 
            axis=1
        )
        


tmp[(tmp.growth_stage_imputed.isna())].to_csv("missing_launchyear_growthstage.csv")

In [ ]:
missing_list = list(tmp[(tmp.growth_stage_imputed.isna())]["name"])

In [ ]:
missing_list

In [ ]:
missing_years = [2012, 2017, 2012, 2015, 2017, 2016, 2016, 2018, 2011, 2016, 2018, 2015, 2015, 2014, 2014, 2013]

In [ ]:
tmp[(tmp.growth_stage_imputed.isna())]

In [ ]:
tmp['growth_stage_imputed']

## GET_DATA

In [3]:
class GetData():

    def get_data():
        data = pd.read_csv("../bpideep/rawdata/data2020-12-03.csv")
        
        #selection of need columns
        data = data[["id", "name", "target", "deep_or_not", "total_funding_source", "employees",
                     "employees_latest", "launch_year", "growth_stage", "linkedin_url"]]
        
        
    
        #imputing the missing launch year (only 33 done manually -> to be automatized if a new source displays it)
        #LAUNCH_YEAR_CLEAN
        impute_missing_launch_year(data)
        
        
        
        # drop 2 duplicated companies Lalilo and Pixyl*
        data.drop(data[(data.id == 1787891) | (data.id == 1893232)].index, inplace = True)
        
        
        
        
        #imputing missing growth_stage (mode of the launch year)
        #GROWTH_STAGE_IMPUTED
        growth_table = get_growth_dict(data)
        
#         import ipdb ; ipdb.set_trace()
        data['growth_stage_imputed'] = data.apply(lambda row: fill_missing_growth(
                growth_table,
                row['growth_stage'], 
                row['launch_year_clean']), 
            axis=1
        )
        
        
        
        #imputing missing employees values from LinkedIn scraping
        #EMPLOYEES_CLEAN
        missing = pd.read_csv("../bpideep/data/missing_employee_count.csv")
        data["employees_clean"] = data.employees_latest
        data.loc[data.name == "CCI Paris Ile de France", "employees_clean"] = 1793

        for url in missing.linkedin_url:
            replace_value = missing[missing.linkedin_url == url]["check"].iloc[0]
            data.loc[data.linkedin_url == url, "employees_clean"] = replace_value
        
        
        
        
        # get the number of patents
        patent = pd.read_csv("../bpideep/data/patents.csv")
        data = pd.merge(data, patent, on= ["id", "id"], how = "left")
        
        return data

#         *    
#                 + 3 duplicated names but with different id : 
#                     1/ Lalilo : 926521 (http://www.lalilo.com/) vs. 1787891 (http://lalilo.fr) 
#             -> same launch date, french website no longer exists + observation  almost filled by NAN + same obs as the .com 
#             -> drop the french Lalilo (1787891)
#                     2/ Pixyl : 892048 vs 1893232 (different websites mentioned but same website page when launched) 
#             -> kept  892048 because more info + the Dealroom profile was verified by Dealroom team on Sept, 1st 2020 vs. pending verification
#                     3/ NANOZ : 1836121 vs 1660543 -> kept both as different companies but the second one is German 

In [4]:
data = GetData.get_data()
X = data.drop(columns = ["target", "deep_or_not"])
y = data.target
y[y ==0.5] = 0

X.columns

<ipython-input-4-b6be683749e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y ==0.5] = 0


Index(['id', 'name', 'total_funding_source', 'employees', 'employees_latest',
       'launch_year', 'growth_stage', 'linkedin_url', 'launch_year_clean',
       'growth_stage_imputed', 'employees_clean', 'nb_patents'],
      dtype='object')

# TEST

In [ ]:
data = GetData.get_data()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data[(data.employees.notna()) & (data.employees_clean.isna())].shape

présence de math.nan ?

In [ ]:
data[(data.employees.isna()) & (data.employees_clean.isna())]

In [ ]:
res = replace_employees(data)

In [ ]:
res[res.employees_imputed.isna()][["name", "employees_latest", "employees", "employees_clean", "employees_imputed", "launch_year_clean"]].groupby(by = "launch_year_clean").count()

In [ ]:
res[(res.employees_imputed.isna()) & (res.launch_year_clean < 2010)][["name", "employees_latest", "employees", "employees_clean", "employees_imputed", "launch_year_clean"]].groupby(by = "launch_year_clean").count()

In [ ]:
yg_median = res[["employees_imputed", "launch_year_clean"]].groupby(by=["launch_year_clean"]).median()
years = data.launch_year_clean.unique()
years = [nb for nb in years if nb >= 2010]

print(yg_median)


for year in years:
    replace_value = yg_median.loc[(year)][0]
    boolean_condition = ((res.launch_year_clean == year) & (res.employees_imputed.isna()))
    res.loc[boolean_condition, "employees_imputed"] = replace_value


In [ ]:
res[res.employees_imputed.isna()][["name", "employees_latest", "employees", "employees_clean", "employees_imputed", "launch_year_clean"]].groupby(by = "launch_year_clean").count()

In [ ]:
#median_bef_2010 
res[res.launch_year_clean < 2010][["employees_imputed"]].median()

In [ ]:
res[res.launch_year_clean.notna()][["employees_imputed"]].median()

# PIPELINE

##### employees imputing  : age

In [5]:
import re
import math
from sklearn.base import BaseEstimator, TransformerMixin


def average_list(range_list):
    return sum(range_list)/len(range_list)

def compute_employees_mean(data):
    range_list = list(data.employees.unique())
    range_list.remove("n.a.")
    keys_list = []
    means_list = []

    for i in range(len(range_list)) :
        if type(range_list[i]) == str :
            temp = re.findall(r'\d+', range_list[i]) 
            res = list(map(int, temp)) 
            mean = average_list(res)
            means_list.append(mean)
            keys_list.append(range_list[i])
        else:
            pass

    zip_iterator = zip(keys_list, means_list)
    range_dict = dict(zip_iterator)
    
    return range_dict

def replace_employees(df):
    df["employees_imputed"] = df.employees_clean
    dictionary = compute_employees_mean(df)
    for key, value in dictionary.items():
        df.loc[(df.employees == key) & (df.employees_latest.isna()), "employees_imputed"] = value
    return df
            
            
class EmployeeImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        
        # impute according to the range mean
#         import ipdb ; ipdb.set_trace()
        data = X.copy()
        data = replace_employees(data)
#         print(data)
        
        
        self.yg_median = data[["employees_imputed", "launch_year_clean"]].groupby(by=["launch_year_clean"]).median()
#         print(self.yg_median)
        self.years = data.launch_year_clean.unique()
        self.years = [nb for nb in self.years if nb >= 2010]
#         print(self.years)
        
#        print(self.yg_median)
        return self
    
    def transform(self, X, y=None):
#         import ipdb ; ipdb.set_trace()
        X = replace_employees(X)
#         print(X)
        
        # impute according to the median of the companies launched after 2010
        for year in self.years:
            replace_value = self.yg_median.loc[(year)][0]
            boolean_condition = ((X.launch_year_clean == year) & (X.employees_imputed.isna()))
            X.loc[boolean_condition, "employees_imputed"] = replace_value
        
#         print("after loop : ", X)
        
        median_all_dataset = X[X.launch_year_clean.notna()][["employees_imputed"]].median()
#         print(X.employees_imputed.isna())
        X.loc[X.employees_imputed.isna(), "employees_imputed"] = replace_value

        return X
    
    



In [6]:
e = EmployeeImputer()
e.fit(X)

EmployeeImputer()

In [7]:
e.transform(X)

,id,name,total_funding_source,employees,employees_latest,launch_year,growth_stage,linkedin_url,launch_year_clean,growth_stage_imputed,employees_clean,nb_patents,employees_imputed
0,1742681,Healthcardionexion.com,0,2-10,8.0,2015.0,seed,https://www.linkedin.com/company/--health,2015.0,seed,8.0,NaN,8.0
1,1743314,4P Pharma,0,11-50,12.0,2014.0,early growth,https://www.linkedin.com/company/4p-pharma,2014.0,early growth,12.0,7.0,12.0
2,1598607,AFYREN,21000000,11-50,31.0,2012.0,early growth,https://www.linkedin.com/company/9419299,2012.0,early growth,31.0,84.0,31.0
3,894817,Abbelight,3250000,11-50,20.0,2016.0,early growth,https://www.linkedin.com/company/abbelight,2016.0,early growth,20.0,NaN,20.0
4,1683057,AblaCare,10000000,2-10,4.0,2017.0,seed,https://www.linkedin.com/company/ablacare,2017.0,seed,4.0,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,1660577,Opta LP,0,NaN,NaN,NaN,NaN,https://fr.linkedin.com/in/opta-lp-9ba2b881,2013.0,early growth,NaN,NaN,14.0
1507,132854,Ownpage Technology,800000,2-10,6.0,2012.0,seed,https://www.linkedin.com/company/ownpage-techn...,2012.0,seed,6.0,NaN,6.0
1508,80695,Per Angusta,4454545,11-50,38.0,2012.0,early growth,https://www.linkedin.com/company/per-angusta,2012.0,early growth,38.0,NaN,38.0
1509,1836415,PowerUp,5000000,11-50,17.0,2017.0,early growth,https://www.linkedin.com/company/powerup-techn...,2017.0,early growth,17.0,3.0,17.0


##### Patent imputing

In [8]:
from sklearn.pipeline import make_pipeline

patent_transformer = make_pipeline(
                                SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 0),
                                RobustScaler())
    
### ROBUSTSCALER or log(1+x) ??

##### Feature  : funding / employees ratio

In [9]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

funding_employees_ratio_constructor = FunctionTransformer(
    lambda df: pd.DataFrame(df["total_funding_source"] / df["employees_imputed"]))


In [10]:
from sklearn.pipeline import Pipeline

ratio_transformer = Pipeline([
    ("imputer1", EmployeeImputer()),
    ("ratio", funding_employees_ratio_constructor),
    ("scaler",  RobustScaler())
])

##### Growth stage OneHotEncoding

In [11]:
from sklearn.preprocessing import OneHotEncoder

cat_growth_stage = OneHotEncoder(handle_unknown='ignore')

In [ ]:
X.growth_stage_imputed

##### Preprocessor

In [12]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ("cat_growth_transformer", cat_growth_stage, ["growth_stage_imputed"]),
    ("ratio_transformer", ratio_transformer, ["employees_latest", "total_funding_source"]),
    ("patent_transformer", patent_transformer, ["nb_patents"])
    ])

In [13]:
from sklearn import set_config; set_config(display='diagram')
preprocessor

ColumnTransformer(transformers=[('cat_growth_transformer',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['growth_stage_imputed']),
                                ('ratio_transformer',
                                 Pipeline(steps=[('imputer1',
                                                  EmployeeImputer()),
                                                 ('ratio',
                                                  FunctionTransformer(func=<function <lambda> at 0x126390310>)),
                                                 ('scaler', RobustScaler())]),
                                 ['employees_latest', 'total_funding_source']),
                                ('patent_transformer',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant')),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['nb_patents'])])

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipemodel = Pipeline(steps=[
                            ('features', preprocessor),
                            ('model', LogisticRegression(C = 1.52, penalty = 'l1', solver= 'liblinear'))
                             ])

In [15]:
pipemodel

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('cat_growth_transformer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['growth_stage_imputed']),
                                                 ('ratio_transformer',
                                                  Pipeline(steps=[('imputer1',
                                                                   EmployeeImputer()),
                                                                  ('ratio',
                                                                   FunctionTransformer(func=<function <lambda> at 0x126390310>)),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['employees_latest',
                                                   'total_funding_source']),
                                                 ('patent_transformer',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['nb_patents'])])),
                ('model',
                 LogisticRegression(C=1.52, penalty='l1', solver='liblinear'))])

In [16]:
pipemodel.fit(X,y)

AttributeError: 'DataFrame' object has no attribute 'employees_clean'

In [ ]:
X.isnull().sum()

In [ ]:
# pd.set_option('display.max_rows', 500)
# check = tmp.growth_stage == tmp.growth_stage_imputed
# tmp["check"] = check
# tmp[(tmp.check == False) & (tmp.growth_stage_imputed.notna())][["name", "growth_stage", "growth_stage_imputed", "check"]]